# Tutorial 1

## Setup

In [ ]:
# sudo pip3 install sklearn pandas imgaug matplotlib numpy

# Zadanie 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import fetch_openml

# MNIST 

Mnist jest to zbiór 70000 pisanych cyfr zapisanych jako obrazki 28px*28px

In [ ]:
mnist = fetch_openml("mnist_784", data_home="./mnist", cache=True)

In [ ]:
plt.imshow(np.array(mnist.data).reshape(70000,28,28)[102])

# FMNIST (Fashion-MNIST)

FMNIST jest to zbiór 70000 zdjęć produktów firmy Zalando jako obrazki 28px*28px

In [ ]:
fmnist = fetch_openml("Fashion-MNIST", data_home="./fmnist", cache=True)

In [ ]:
plt.imshow(np.array(fmnist.data).reshape(70000,28,28)[102])

# CIFAR_10

CIFAR_10 jest to zbiór 60000 obrazków 32px*32px (6000 na klasę) w 10 różnych klasach (samolot, samochód, ptak, kot, jeleń, pies, żaba, koń, statek, ciężarówka)

In [ ]:
cifar10 = fetch_openml("CIFAR_10", data_home="./cifar10", cache=True)

In [ ]:
plt.imshow(np.transpose(np.array(cifar10.data).reshape(60000,3,32,32)[4].astype(int), (1,2,0)))

# SmallNorb

SmallNorb to zbiór obrazów do eksperymentów z rozpoznawaniem obrazów 3D. Składa się z 50 zabawek należących do 5 różnych kategorii. Dla każdej zabawki jest to zestaw zdjęć pod różnymi warunkami światła, podniesieniami i azymutami

![smallNorb.png](./smallNorb.png)

# TNG

TNG to zbiór wszystkich odcinków Star Trek TNG. Każdy rząd zawiera osobną wypowiedź albo opis ze scenariusza.

In [ ]:
tng = pd.read_csv("./TNG.csv")

In [ ]:
tng

# Zadanie 2

## Communities and Crime Data Set

In [ ]:
crime = pd.read_csv("./crime.csv")

In [ ]:
crime

# Wybieranie danych

### Wybranie `n` pierwszych

In [ ]:
crime[:400]

### Wybranie `n` losowych

In [ ]:
crime.sample(400)

### Podział danych testowych z użyciem sklearn

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(crime.drop(columns='nonViolPerPop'), crime['nonViolPerPop'], test_size=0.2)
X_test

# Normalizacja

In [ ]:
crime_replaced = crime.set_index(["communityname", "state", "countyCode", "communityCode", "fold"]).replace('?', np.NaN)
crime_replaced

### Uzupełnienie brakujących danych

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
crime_filled = pd.DataFrame(imputer.fit_transform(crime_replaced),columns=crime_replaced.columns,index=crime_replaced.index)
crime_filled

### Skalowanie z użyciem wartości maksymalnej i minimalnej.

![minmax.png](./minmax.png)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
crime_scaled = pd.DataFrame(min_max_scaler.fit_transform(crime_filled),columns=crime_filled.columns,index=crime_filled.index)
crime_scaled

In [ ]:
crime_scaled.describe()

### Standaryzacja - skalowanie mające na celu przesunięcie rozkładu tak, aby średnia była równa 0, a odchylenie standardowe 1.

![standarization.png](./standarization.png)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
crime_standardized = pd.DataFrame(scaler.fit_transform(crime_filled),columns=crime_filled.columns,index=crime_filled.index)
crime_standardized

In [ ]:
crime_standardized.describe()

## Normalizacja - przeskalowanie każdej obserwacji do długości 1

![normalization.png](./normalization.png)

In [ ]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
crime_normalized = pd.DataFrame(normalizer.fit_transform(crime_filled),columns=crime_filled.columns,index=crime_filled.index)
crime_normalized

# Zadanie 3

# k-NN

Algorytm k-NN:
 * Liczymy odległość wszystkich pomiarów od rozważanego
 * Wybieramy k najbliższych pomiarów do naszego
 * Przypisujemy rozważanemu pomiarowi taką etykietę, jaką ma najwięcej pomiarów z tych k wybranych
 
![kNN.png](./kNN.png)

### k-NN z użyciem sklearn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(mnist.data.reshape((70000, 28*28))[:500], mnist.target[:500])

In [ ]:
neigh.predict(mnist.data.reshape((70000, 28*28))[1000:1005])

In [ ]:
plt.imshow(mnist.data.reshape((70000, 28, 28))[1002])

Implementacje KNeighborsClassifier(parametr `algorithm` konstruktora):
 * brute - dobry dla danych rzadkich, za każdym razem liczy odległości do wszystkich punktów
 * ball_tree i kd_tree - struktury reprezentujące podział wieliwymiarowych przestrzeni (mają strukturę drzew binarnych, w których każdy element, który nie jest liściem, można interpretować jako podział przestrzeni na dwie części). Mają podobne benchmarki. (Ball Tree zwykle nieco szybciej się uczy, natomiast k-d Tree szybciej oblicza predykcje)
 * auto - ustala najlepszy algorytm dla podanych danych i go stosuje

### Biblioteka FAISS https://github.com/facebookresearch/faiss

Biblioteka stosowana do szybkiego rozwiązywania problemów wyszykiwania podobieństwa i grupowania. Napisana głównie w C++, z dowiązaniami do Pytohna. Najprzydatniejsze algorytmy są zaimplementowanie do użycia na GPU z wykorzystaniem CUDA.

# Zadanie 4

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.datasets import load_iris, load_wine
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier, KernelDensity
from sklearn.model_selection import KFold,train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
iris = load_iris()
wines = load_wine()
rnc = KNeighborsClassifier()

## Wstęp teoretyczny

**Cross-validation** to metoda oceny modelu statystycznego. Jest lepsza od pozostałych metod, ponieważ zwraca informacje o tym, jak dobrze zachowa się nauczona maszyna podczas klasyfikacji nowych elementów. Polega na tym, że nie wykorzystuje się całego zbioru danych do nauki. Należy podzielić zbiór na dwie części: jedną wykorzystać do nauki, a drugą do testu nauczonej maszyny. Test weryfikuje, jak dobrze nauczona maszyna klasyfikuje elementy.

Istnieją różne ulepszenia bazowej idei kroswalidacji. Jedną z nich jest **K-fold cross-validation**. Zamiast dzielić zbiór na dwie części, dzielimy go na K części. Wykonujemy procedurę nauki K razy, za każdym razem wybieramy jeden z K podzbiorów jako zbiór testowy, natomiast pozostałe K-1 zbiorów są użyte do nauki. Następnie obliczamy średnią z wszystkich testów, co daje nam ocenę.

Przypadkiem szczególnym K-fold cross-validation jest metoda **leave-one-out**. W tym przypadku K jest równe liczności zbioru danych, co powoduje, że używamy do nauki wszystkich elementów poza jednym, a następnie przeprowadzamy test tylko na jednym elemencie. Ze wszystkich N testów wyciągamy średnią. Na początku wydaje się to niewydajną metodą, ponieważ należy przeprowadzić algorytm uczenia N razy, jednak istnieją usprawnienia, które sprawiają, że przeprowadzenie ewaluacji nie jest tak kosztowne.

## Cross-validation - wariant podstawowy

In [ ]:
results = cross_val_score(rnc, iris.data, iris.target, cv = 2)
print("Liczba iteracji: ", len(results))
print("Średnia trafność: {:.2f}".format(results.mean()))

## K-fold cross-validation

In [ ]:
kfold = KFold(n_splits=5)
results = cross_val_score(rnc, iris.data, iris.target, cv = kfold)
print("Liczba iteracji: ", len(results))
print("Średnia trafność: {:.2f}".format(results.mean()))

## Cross-validation leave-one-out

In [ ]:
loo = LeaveOneOut()

results = cross_val_score(rnc, wines.data, wines.target, cv = loo)
print("Liczba iteracji: ", len(results))
print("Średnia trafność: {:.2f}".format(results.mean()))

# Zadanie 5

## k-NN

### K = 3

In [ ]:
lenghts = []
results = []
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
for i in range(len(X_train) - 2):
    x_train_trimmed = X_train[i:][:]
    y_train_trimmed = y_train[i:][:]
    nc = KNeighborsClassifier(n_neighbors = 3)
    model = nc.fit(x_train_trimmed, y_train_trimmed)
    lenghts += [len(x_train_trimmed)]
    results += [model.score(X_test, y_test)]
plt.plot(lenghts, results)
plt.ylabel("Precyzja klasyfikacji")
plt.xlabel("Ilość elementów w zbiorze uczącym")
plt.show()

### K = 5

In [ ]:
lenghts = []
results = []
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
for i in range(len(X_train) - 4):
    x_train_trimmed = X_train[i:][:]
    y_train_trimmed = y_train[i:][:]
    nc = KNeighborsClassifier(n_neighbors = 5)
    model = nc.fit(x_train_trimmed, y_train_trimmed)
    lenghts += [len(x_train_trimmed)]
    results += [accuracy_score(y_test, nc.predict(X_test))]
plt.plot(lenghts, results)
plt.ylabel("Precyzja klasyfikacji")
plt.xlabel("Ilość elementów w zbiorze uczącym")
plt.show()

### K = 1

In [ ]:
lenghts = []
results = []
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
for i in range(len(X_train)):
    x_train_trimmed = X_train[i:][:]
    y_train_trimmed = y_train[i:][:]
    nc = KNeighborsClassifier(n_neighbors = 1)
    model = nc.fit(x_train_trimmed, y_train_trimmed)
    lenghts += [len(x_train_trimmed)]
    results += [accuracy_score(y_test, nc.predict(X_test))]
plt.plot(lenghts, results)
plt.ylabel("Precyzja klasyfikacji")
plt.xlabel("Ilość elementów w zbiorze uczącym")
plt.show()

### Zmiana k

In [ ]:
results = defaultdict(list)
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
for j in range(1000):
    for i in range(1,20,1):
        x_train_trimmed = X_train[:40][:]
        y_train_trimmed = y_train[:40][:]
        nc = KNeighborsClassifier(n_neighbors = i)
        model = nc.fit(x_train_trimmed, y_train_trimmed)
        results[i] += [accuracy_score(y_test, nc.predict(X_test))]
for k,v in results.items():
    results[k] = sum(v) / len(v)
results
plt.plot(list(results.keys()), list(results.values()))
plt.ylabel("Precyzja klasyfikacji")
plt.xlabel("K")
plt.show()

### Zmiana promienia

In [ ]:
results = defaultdict(list)
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
for j in range(100):
    for i in range(1,20,1):
        x_train_trimmed = X_train[:40][:]
        y_train_trimmed = y_train[:40][:]
        nc = RadiusNeighborsClassifier(radius = i)
        model = nc.fit(x_train_trimmed, y_train_trimmed)
        results[i] += [accuracy_score(y_test, nc.predict(X_test))]
for k,v in results.items():
    results[k] = sum(v) / len(v)
results
plt.plot(list(results.keys()), list(results.values()))
plt.ylabel("Precyzja klasyfikacji")
plt.xlabel("Radius")
plt.show()

### Zmiana algorytmu

In [ ]:
results = defaultdict(list)
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
for j in range(100):
    for i in ['gaussian', 'tophat', 'epanechnikov', 'exponential','linear','cosine']:
        x_train_trimmed = X_train[:100][:]
        y_train_trimmed = y_train[:100][:]
        nc = KernelDensity(kernel = i)
        model = nc.fit(x_train_trimmed, y_train_trimmed)
        results[i] += [nc.score(X_test)]
for k,v in results.items():
    results[k] = sum(v) / len(v)
print(results)
plt.plot(list(results.keys()), list(results.values()))
plt.ylabel("Precyzja klasyfikacji")
plt.xlabel("Kernel")
plt.show()

# Zadanie 6

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt

In [ ]:
# wybieramy mały podzbiór - 100 elementów - danych ze zbioru cifar-10
_, subsample_x, _, subsample_y = train_test_split(cifar10.data, cifar10.target, test_size=0.0016, stratify=cifar10.target)

In [ ]:
# funkcje pomocnicze
def img_from_array(arr):
    return arr.reshape(3, 32, 32).transpose(1,2,0).astype('uint8')

def img_to_array(img):
    return img.transpose(2,0,1).reshape(3 * 32 * 32)

def display_imgs(*images):
    f = plt.figure()
    ind = 1
    for img in images:
        f.add_subplot(1, 2, ind)
        plt.imshow(img)
        ind += 1

example = img_from_array(subsample_x[0])
plt.imshow(example)

In [ ]:
# możemy powiększyć zbiór danych np. odwracając obraz, nakładając szum...
flipped_img = np.fliplr(example) # odwracanie horyzontalne

def apply_noise(img): # gaussian noise
    mean, var = 0, 50
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,img.shape)
    return np.clip(gauss + img, 0, 255).astype(int)

noisy_img = apply_noise(example)

display_imgs(flipped_img, noisy_img)

In [ ]:
# procedura losowo powiększająca zbiór danych
# z zadanym prawdopodobieństwem dodajemy zmodyfikowane obrazy do zbioru

def augment(data, labels,flip_chance=0.2, noise_chance=0.1):
    aug_data, aug_labels = [], []
    for img, label in zip(data, labels):
        new_img = None
        if np.random.rand() < flip_chance:
            new_img = np.fliplr(img_from_array(img))
        if np.random.rand() < noise_chance:
            new_img = apply_noise(new_img) if new_img is not None else apply_noise(img_from_array(img))

        if new_img is not None:
            aug_data.append(img_to_array(new_img))
            aug_labels.append(label)

    return (np.append(data, aug_data, 0), np.append(labels, aug_labels, 0))

augmented_x, augmented_y = augment(subsample_x, subsample_y)

print(f"Added {len(augmented_x) - len(subsample_x)} new images")

## Modelujemy zachowanie miary jakości przykładowego klasyfikatora kolejno powiększając zbiór danych

In [ ]:
from sklearn.model_selection import cross_val_score

iters = 10
scores = []
data, labels = subsample_x, subsample_y
original_size = len(data)
for i in range(iters):
    clf = KNeighborsClassifier()
    res = cross_val_score(clf, data, labels)
    scores.append((np.mean(res), len(data) - original_size))
    data, labels = augment(data, labels)

sc, num = list(zip(*scores))
fig, ax = plt.subplots()
ax.set_xlabel('Number of augmented images')
ax.set_ylabel('Cross validation score')
ax.plot(num, sc, 'o')
plt.show()
print(f"Original dataset size: {original_size}")

# Zadanie 7
### pełny spis dostępnych augmentatorów na https://imgaug.readthedocs.io/en/latest/source/overview_of_augmenters.html
### git biblioteki https://github.com/aleju/imgaug/blob/master/README.md

In [ ]:
from imgaug import augmenters as augs

images = subsample_x.reshape(len(subsample_x), 3, 32, 32).transpose(0, 2, 3, 1).astype(np.uint8)

In [ ]:
augmenters = [
    augs.Fliplr(1), # horyzontalne odwrócenie obrazu
    augs.CropAndPad(percent=(0, 0.1)), # przycięcie obrazu losowo 0-10% rozmiaru z dodanym paddingiem 
    augs.Rotate((-15, 15)), # obrócenie obrazu o -15 - 15 stopni
    augs.Crop(px=(1, 16), keep_size=True), # przycięcie zdjęcia od 1 do 16 pikselów
    augs.GaussianBlur(sigma=(0, 1.0)), # tak jak nazwa mówi
    augs.Multiply((0.8, 1.2), per_channel=0.5), # zmiana jasności obrazu 80% - 120%
    augs.EdgeDetect(alpha=(0, 0.7)), # wykrywanie krawędzi
    augs.Affine(scale=(0.8, 1.2)) # skalowanie obrazu 80% - 120% 
]





In [ ]:
# sekwencyjne aplikowanie augmentatorów z losową kolejnością
seq = augs.Sequential(augmenters, random_order=True)
image_aug = seq(images=images)

display_imgs(images[0], image_aug[0])

In [ ]:
# aplikowanie 1-2 losowych augmentatorów
some = augs.SomeOf((1, 2), augmenters)
image_aug = some(images=images)

display_imgs(images[0], image_aug[0])

In [ ]:
# aplikowanie na 30% obrazów
sometimes = augs.Sometimes(0.3, augmenters)
image_aug = sometimes(images=images)

display_imgs(images[0], image_aug[0])

In [ ]:
# aplikowanie jednego z augmentatorów
one = augs.OneOf(augmenters)
image_aug = one(images=images)

display_imgs(images[0], image_aug[0])

# 8. Zadanie do wykonania

Wykonać punkty 6 i 7 na zbiorach MNIST, FMNIST przy pomocy imgaug oraz tworząc
obrazki dodatkowe poprzez zaburzenie danych (np. dla x% pikseli losować liczbę 0,1 (MNIST), i
odpowiednią – stopień szarości - dla zbioru FMNIST)

* Wczytaj dane MNIST, FMINST
* Podziel dane na treningowe i testowe
* Stwórz klasyfikator
* Zbadaj dokładność klasyfikatora
* Dokonaj augmentacji danych - imgaug, własna procedura
* Porównaj miary jakości klasyfikatora